# Wire-scanner errors 

This notebook investigates the accuracy of the estimated beam moments from the wire-scanners. We approximate the wire-scanner profile as a histogram. There is then uncertainty in the $i^{th}$ wire-scanner position $x_i$ and signal height $f_i$. 

In [ ]:
import sys

import numpy as np
import matplotlib.pyplot as plt
import proplot as plot
from tqdm import tqdm, trange

sys.path.append('..')
from tools import beam_analysis as ba
from tools import plotting as myplt
from tools import utils
from tools.accphys_utils import V_matrix_4x4_uncoupled
from tools.accphys_utils import rotation_matrix_4D

plot.rc['grid.alpha'] = False
plot.rc['axes.grid'] = False

In [ ]:
def bin_data(data, n_steps, lims):
    counts, bin_edges = np.histogram(data, n_steps, lims)
    delta = np.mean(np.diff(bin_edges))
    positions = (bin_edges + 0.5 * delta)[:-1]
    return positions, counts

In [ ]:
def variance(positions, counts):
    N = np.sum(counts)
    x_avg = np.sum(positions * counts) / (N - 1)
    x2_avg = np.sum(positions**2 * counts) / (N - 1)
    return x2_avg - x_avg**2 

In [ ]:
def get_sig_xy(sig_xx, sig_yy, sig_uu):
    """Get cov(x, y) assuming u axis is 45 degrees above x axis."""
    sn = cs = np.sqrt(0.5)
    sig_xy = (sig_uu - sig_xx*(cs**2) - sig_yy*(sn**2)) / (2 * sn * cs)
    return sig_xy

In [ ]:
def set_twiss(X, alpha_x_target, alpha_y_target, beta_x_target, beta_y_target):
    # Normalize with beam Twiss parameters.
    Sigma = np.cov(X.T)
    Sigma = Sigma[:4, :4]
    alpha_x, alpha_y, beta_x, beta_y = ba.twiss2D(Sigma)
    V = V_matrix_4x4_uncoupled(alpha_x, alpha_y, beta_x, beta_y)
    X = utils.apply(np.linalg.inv(V), X)
    # Unnormalize with desired parameters.
    V = V_matrix_4x4_uncoupled(alpha_x_target, alpha_y_target, beta_x_target, beta_y_target)
    X = utils.apply(V, X)
    return X

## Setup

SNS wire-scanner parameters.

In [ ]:
n_steps = 90
ulims = [-133.5, 133.5] # [mm]
xlims = ylims = [ulim / np.sqrt(2) for ulim in ulims]

The following is a simulated distribution at the RTBT entrance.

In [ ]:
X = np.loadtxt('./scan/_input/init_dist_128K.dat')
X = X[:, :4]
X *= 1000.0

We're actually going to change the Twiss parameters to what is seen at WS24.

In [ ]:
# Typical Twiss parameters at WS24
alpha_x_target = 0.67 # [rad]
alpha_y_target = -1.48 # [rad]
beta_x_target = 6.8 # [m/rad]
beta_y_target = 14.9 # [m/rad]

X = set_twiss(X, alpha_x_target, alpha_y_target, beta_x_target, beta_y_target)

# Save new covariance matrix.
Sigma = np.cov(X.T)
Sigma = Sigma[:4, :4]
alpha_x, alpha_y, beta_x, beta_y = ba.twiss2D(Sigma)
print('alpha_x = {} [rad]'.format(alpha_x))
print('alpha_y = {} [rad]'.format(alpha_y))
print('beta_x = {} [rad]'.format(beta_x))
print('beta_y = {} [rad]'.format(beta_y))

Store the new covariance matrix.

In [ ]:
Sigma = np.cov(X.T)
xx = X[:, 0]
yy = X[:, 2]
uu = np.sqrt(0.5) * (xx - yy)
sig_xx_true = Sigma[0, 0]
sig_yy_true = Sigma[2, 2]
sig_xy_true = Sigma[0, 2]
sig_uu_true = np.var(uu)

In [ ]:
print('Sigma =')
print(Sigma)
print('Corr =')
print(utils.cov2corr(Sigma))

In [ ]:
myplt.corner(X, bins=40, kind='hist', pad=0, cmap='blues');

Estimate the moments along x, y, and u from the binned data. This will be used as the "true" value. (The error caused by binning will be handled later.)

In [ ]:
pos_x, counts_x = bin_data(xx, n_steps, xlims)
pos_y, counts_y = bin_data(yy, n_steps, ylims)
pos_u, counts_u = bin_data(uu, n_steps, ulims)
sig_xx_est = variance(pos_x, counts_x)
sig_yy_est = variance(pos_y, counts_y)
sig_uu_est = variance(pos_u, counts_u)
sig_xy_est = get_sig_xy(sig_xx_est, sig_yy_est, sig_uu_est)
corr_xy_est = sig_xy_est / np.sqrt(sig_xx_est * sig_yy_est)

print('Estimated from histograms')
print('-------------------------')
print('sig_xx = {:.3f} [mm^2]'.format(sig_xx_est))
print('sig_yy = {:.3f} [mm^2]'.format(sig_yy_est))
print('sig_uu = {:.3f} [mm^2]'.format(sig_uu_est))
print('sig_xy = {:.3f} [mm^2]'.format(sig_xy_est))
print('corr_xy = {:.3f}'.format(corr_xy_est))
print()
print('Actual')
print('------')
print('sig_xx = {:.3f} [mm^2]'.format(sig_xx_true))
print('sig_yy = {:.3f} [mm^2]'.format(sig_yy_true))
print('sig_uu = {:.3f} [mm^2]'.format(sig_uu_true))
print('sig_xy = {:.3f} [mm^2]'.format(sig_xy_true))
print('corr_xy = {:.3f}'.format(corr_xy_est))

In [ ]:
fig, axes = plot.subplots(ncols=3, figsize=(8, 2), sharex=False)
plt_kws = dict(color='black', marker='.', ms=None, lw=0)
axes[0].plot(pos_x, counts_x, **plt_kws)
axes[1].plot(pos_y, counts_y, **plt_kws)
axes[2].plot(pos_u, counts_u, **plt_kws)
axes[0].set_xlabel('x [mm]')
axes[1].set_xlabel('y [mm]')
axes[2].set_xlabel('u [mm]')
axes[0].set_ylabel('Counts');

## Uncertainty in estimated variance.

One source of uncertainty is the signal height at each wire position. 

In [ ]:
def noisy_counts(counts, rms_frac_err=0.):
    noise = np.random.normal(scale=rms_frac_err, size=len(counts))
    counts = counts * (1.0 + noise)
    counts = counts.astype(int)
    counts = np.clip(counts, 0, None)
    return counts

In [ ]:
pos = pos_y
counts = np.copy(counts_y)

In [ ]:
rms_frac_count_err = 0.1
_counts = noisy_counts(counts, rms_frac_count_err)

fig, ax = plot.subplots(figsize=(6, 1.5))
ax.plot(pos, counts, color='k')
ymin, ymax = ax.get_ylim()
ax.plot(pos, _counts, color='red8', marker='.')
ax.set_ylim(ymin, 1.2 * ymax)
ax.legend(labels=['original', 'noisy'], ncol=1);

Another source of uncertainty is the wire-scanner position.

In [ ]:
def noisy_positions(positions, rms_err):
    noise = np.random.normal(scale=rms_err, size=len(positions))
    return positions + noise

In [ ]:
rms_pos_err = 0.5 # [mm]

fig, ax = plot.subplots(figsize=(6, 1.5))
ax.plot(pos, counts, color='k')
ymin, ymax = ax.get_ylim()
ax.plot(noisy_positions(pos, rms_pos_err), counts, color='red8', marker='.')
ax.set_ylim(ymin, 1.2 * ymax)
ax.legend(labels=['original', 'noisy'], ncol=1);

In [ ]:
def run_monte_carlo(pos, counts, rms_pos_err, rms_frac_count_err, n_trials):
    sigs = []
    for _ in range(n_trials):
        pos_with_noise = noisy_positions(pos, rms_pos_err)
        counts_with_noise = noisy_counts(counts, rms_frac_count_err)
        sigs.append(variance(pos_with_noise, counts_with_noise))
    return sigs

In [ ]:
n_trials = 10000
rms_pos_err = 0.5 # [mm]
rms_frac_count_err = 0.05

sigs_x = run_monte_carlo(pos_x, counts_x, rms_pos_err, rms_frac_count_err, n_trials)
sigs_y = run_monte_carlo(pos_y, counts_y, rms_pos_err, rms_frac_count_err, n_trials)
sigs_u = run_monte_carlo(pos_u, counts_u, rms_pos_err, rms_frac_count_err, n_trials)

fig, axes = plot.subplots(nrows=2, ncols=3, figsize=(7, 3), sharex=False)
hist_kws = dict(histtype='stepfilled', color='black', bins='auto', alpha=0.2)
for i in range(3):
    sigs = [sigs_x, sigs_y, sigs_u][i]
    sig_est = [sig_xx_est, sig_yy_est, sig_uu_est][i]
    dim = ['x', 'y', 'u'][i]
    sizes = np.sqrt(sigs)
    size_est = np.sqrt(sig_est)
    axes[0, i].hist(sizes, **hist_kws)
    axes[0, i].axvline(size_est, c='r')
    axes[0, i].axvline(np.mean(sizes), c='b')
    axes[0, i].format(title='{} wire'.format(dim), xlabel=r'$\sigma_{}$ [mm]'.format(dim))
    axes[0, i].annotate('mean = {:.2f}'.format(np.mean(sizes)), xy=(0.7, 0.85), xycoords='axes fraction', fontsize=6)
    axes[0, i].annotate('std = {:.2f}'.format(np.std(sizes)), xy=(0.7, 0.75), xycoords='axes fraction', fontsize=6)
    axes[0, i].annotate('true = {:.2f}'.format(size_est), xy=(0.7, 0.65), xycoords='axes fraction', fontsize=6)
    axes[1, i].hist(sigs, **hist_kws)
    axes[1, i].axvline(sig_est, c='r')
    axes[1, i].axvline(np.mean(sigs), c='b')
    axes[1, i].format(xlabel=r'$\sigma_{}^2$ [mm^2]'.format(dim))
    axes[1, i].annotate('mean = {:.2f}'.format(np.mean(sigs)), xy=(0.7, 0.85), xycoords='axes fraction', fontsize=6)
    axes[1, i].annotate('std = {:.2f}'.format(np.std(sigs)), xy=(0.7, 0.75), xycoords='axes fraction', fontsize=6)
    axes[1, i].annotate('true = {:.2f}'.format(sig_est), xy=(0.7, 0.65), xycoords='axes fraction', fontsize=6)
axes[0, 2].legend(labels=['true', 'mean'], ncol=1, loc=(1.02, 0), fontsize='small', handlelength=1.5);

The calculation seems very tolerant to noisy profiles as long as the noise is asymmetric about the beam center. Might need to look into this... for example, one wire or a few wires could be more noisy than the others. But our actual profiles look nice and smooth.

## Uncertainty in tilted wire angle 

The calculation of $\langle{xy}\rangle$ from $\langle{xx}\rangle$, $\langle{yy}\rangle$, and $\langle{xy}\rangle$ depends on $\phi$, the angle of the $u$ axis above the $x$ axis. Here we assume the other moments are measured perfectly, but that there is some error in angle.

In [ ]:
phi = np.radians(45.0)
dphi = np.radians(2.0)

corr_xy_list = []
for _ in trange(1000):
    phi_err = phi + np.random.uniform(-0.5 * dphi, 0.5 * dphi)
    uu = xx * np.cos(phi_err) + yy * np.sin(phi_err)
    sig_uu = variance(*bin_data(uu, n_steps, ulims))
    sig_xx = sig_xx_est
    sig_yy = sig_yy_est
    sig_xy = get_sig_xy(sig_xx, sig_yy, sig_uu)
    corr_xy_list.append(sig_xy / np.sqrt(sig_xx * sig_yy))
    
fig, ax = plot.subplots(figsize=(4, 2))
ax.hist(corr_xy_list, histtype='stepfilled', color='black')
ax.format(xlabel='corr. coeff.')

There is really not much affect from the tilt angle.

## Effect of measurement errors on $\langle{xy}\rangle$ calculation

In [ ]:
n_trials = 50000
frac_variance_errs = np.linspace(0.01, 0.1, 10)

In [ ]:
fig, axes = plot.subplots(nrows=len(frac_variance_errs), figsize=(5, 12))
axes[0].set_title(r'Calculated $r_{xy}$ with errors in $\sigma_x$, $\sigma_y$, $\sigma_u$ ({:.0f}%)')

stds = []

for frac_variance_err, ax in zip(frac_variance_errs, axes):
    corr_xy_list = []
    for _ in range(n_trials):
        sig_uu = sig_uu_est * (1.0 + np.random.normal(scale=frac_variance_err))
        sig_xx = sig_xx_est * (1.0 + np.random.normal(scale=frac_variance_err))
        sig_yy = sig_yy_est * (1.0 + np.random.normal(scale=frac_variance_err))
        sig_xy = get_sig_xy(sig_xx, sig_yy, sig_uu)
        corr_xy_list.append(sig_xy / np.sqrt(sig_xx * sig_yy))
    mean = np.mean(corr_xy_list)
    std = np.std(corr_xy_list)
    stds.append(std)
    
    ax.hist(corr_xy_list, bins='auto', histtype='stepfilled', color='black', alpha=0.2)
    ax.annotate(r'Frac. $\sigma$ error = {:.0f}%'.format(100 * frac_variance_err),
                xy=(0.02, 0.85), xycoords='axes fraction')
    ax.annotate('true = {:.3f}'.format(corr_xy_est), xy=(0.8, 0.85), 
                xycoords='axes fraction', fontsize='small')
    ax.annotate('mean = {:.3f}'.format(mean), xy=(0.8, 0.75), 
                xycoords='axes fraction', fontsize='small')
    ax.annotate('stdev = {:.3f}'.format(std), xy=(0.8, 0.65), 
                xycoords='axes fraction', fontsize='small')
    ax.axvline(corr_xy_est, c='r')
    ax.axvline(mean, c='b')
axes[0].legend(labels=['true', 'mean'], loc=(1.02, 0), ncol=1, fontsize='small');

The standard deviation in cov(x, y) can be written exactly:

$$
\delta\sigma_{xy} = \sqrt{
    (\delta \sigma_{uu})^2 + \frac{1}{4}(\delta\sigma_{xx})^2 + \frac{1}{4}(\delta\sigma_{yy})^2
    + (\frac{\partial \sigma_{xy}}{\partial \phi} \delta\phi)^2
} 
$$

Assuming the uncertainty along each dimension is the same, and assuming there is no angle uncertainty, we have 

$\delta \sigma_{xy} = \sqrt{\frac{3}{2}} \delta \sigma_{uu}$

In [ ]:
slope = (stds[-1] - stds[0]) / (frac_variance_errs[-1] - frac_variance_errs[0])
print('slope =', slope)
print('sqrt(3 / 2) =', np.sqrt(3 / 2))

fig, ax = plot.subplots()
ax.scatter(frac_variance_errs, stds)
ax.format(xlabel='RMS fractional error in measured variance',
          ylabel='Standard deviation in measured cov(x, y)')

The dependence on angle can be found by taking derivatives. It will get complicated when plugging in to the emittance calculation since it is nonlinear, so best to do it numerically.

## What about different beam shapes? 

We have a square beam, which is not rotationally symmetric. This seems like it would affect the accuracy of the calculated cov(x, y). But maybe not. 

In [ ]:
corr_xy_true_list = []
corr_xy_est_list = []
angles = np.radians(np.linspace(0, 180.0, 12))

for angle in tqdm(angles):
    # Rotate the distribution.
    Xrot = np.copy(X)
    Xrot[:, 0] *= 0.5
    Xrot = utils.apply(rotation_matrix_4D(-angle), Xrot)
    # Record the true cov(x, y).
    Sigma = np.cov(Xrot.T)
    corr_xy_true_list.append(Sigma[0, 2] / np.sqrt(Sigma[0, 0] * Sigma[2, 2]))
    # Estimate cov(x, y) from the histograms along x, y and u.
    xx = Xrot[:, 0]
    yy = Xrot[:, 2]
    uu = np.sqrt(0.5) * (xx + yy)
    pos_x, counts_x = bin_data(xx, n_steps, xlims)
    pos_y, counts_y = bin_data(yy, n_steps, ylims)
    pos_u, counts_u = bin_data(uu, n_steps, ulims)
    sig_xx_est = variance(pos_x, counts_x)
    sig_yy_est = variance(pos_y, counts_y)
    sig_uu_est = variance(pos_u, counts_u)
    f = 0.05
    sig_xx_est *= np.random.uniform(1 - f, 1 + f)
    sig_yy_est *= np.random.uniform(1 - f, 1 + f)
    sig_uu_est *= np.random.uniform(1 - f, 1 + f)
    sig_xy_est = get_sig_xy(sig_xx_est, sig_yy_est, sig_uu_est)
    corr_xy_est_list.append(sig_xy_est / np.sqrt(sig_xx_est * sig_yy_est))

In [ ]:
fig, ax = plot.subplots(figsize=(4, 2))
ax.plot(np.degrees(angles), corr_xy_est_list, marker='|', color='red8')
ax.plot(np.degrees(angles), corr_xy_true_list, marker='.', color='blue8')
ax.legend(labels=['est', 'true'], ncols=1, fontsize='small')
ax.format(xformatter='deg', xlabel='Beam tilt angle', ylabel='corr(x, y)')

## Effect of beam size on uncertainty in measured moments